In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pprint

In [2]:
#set up ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\David\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [3]:
#set url to first website
url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [4]:
#visit website through browser and convert to bs
browser.visit(url)
html_1=browser.html
soup=bs(html_1,'html.parser')

In [ ]:
#print soup to check that the parser included everything needed
print(soup)

In [6]:
#find title and preview paragraph for latest news story
title = soup.find_all('div',class_="content_title")[1].text
news_p = soup.find('div', class_="article_teaser_body").text

In [18]:
#check to make sure title and news_p are correct
print(title)
print(news_p)

Tricky Terrain: Helping to Assure a Safe Rover Landing
How two new technologies will help Perseverance, NASA’s most sophisticated rover yet, touch down onto the surface of Mars this month.


In [9]:
#set url_2 to second website and set up with browser
url_2='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url_2)
html_2=browser.html

In [10]:
#find image url with soup
soup_2=bs(html_2,"html.parser")
image_url=soup_2.find_all('img')[1]["src"]
featured_image='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + image_url
#check for correct url
print(featured_image)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


In [11]:
#set url for 3rd website
url_3="https://space-facts.com/mars/"
#use read html to find tables within website
tables=pd.read_html(url_3)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [12]:
#extract table needed and clean for use
df=tables[0]
df.columns=['Category','Mars']
df.set_index("Category",inplace=True)
df

,Mars
Category,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [13]:
#convert back to html
df=df.to_html("mars_facts.html",border=1)

In [14]:
#define blank list that will house image dictionaries
hemisphere_image_urls=[]
#define list that will be used in the click function loop
click_lst=['Cerberus','Schiaparelli','Syrtis','Valles']
#define path that will be the first part of img_url
regular_path='https://astrogeology.usgs.gov'

In [15]:
#set url for fourth website
url_4='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
#create loop that will click through the emelemts in click_list 
#and return title of image and image url
for x in range(4):
    browser.visit(url_4)
    browser.links.find_by_partial_text(click_lst[x]).click()
    html_3=browser.html
    soup_3=bs(html_3,'html.parser')
    titles=soup_3.find('h2',class_="title").text
    img_url=soup_3.find_all('img',class_="wide-image")[0]['src']
    #append results to list
    hemisphere_image_urls.append({"title":titles,"img_url":regular_path + img_url})

In [16]:
#quit out of browser
browser.quit()

In [17]:
#check to make sure correct data was collected
pprint.pprint(hemisphere_image_urls)

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]
